# Inclusión de variables categóricas en los modelos

Por último analizaremos el impacto de incluir variables categóricas en nuestros modelos.

Esta es una pregunta bastante común en problemas de modelado. ¿Qué tanto cambia el resultado a través de diferentes categorías?
Las categorías son variables discretas sin un orden específico. Por ejemplo:

- Sexo: masculino, femenino.
- Continente: América, Asia, África.
- Universidad: ITESO, Tecnológico de Monterrey, UDG.

Hay que tener cuidado, ya que en muchos contextos, este tipo de variables se codifican como variables numéricas (one-hot encoding, label encoding, entre otros) y se suelen tratar en los modelos como variables continuas, lo cual puede llevar a conclusiones incorrectas.

Una idea un poco menos común es que los parámetros del modelo dependan de las categorías. En este caso, podríamos decir que desarrollamos un modelo por cada categoría. De esta forma, no se sufren de los problemas antes mencionados, y es lo que estudiaremos en este cuaderno.

Usaremos todo lo que vimos en el cuaderno anterior:

In [ ]:
# Importar pandas
import pandas as pd
# Importar pymc
import pymc as pm
# Importar arviz
import arviz as az
# Importar numpy
import numpy as np
# Importar train_test_split
from sklearn.model_selection import train_test_split
# Importamos precision_recall_curve y f1_score
from sklearn.metrics import precision_recall_curve, f1_score
# Importamos pyplot
from matplotlib import pyplot as plt
# Importamos seaborn
import seaborn as sns

In [ ]:
# Leer el archivo de datos (bank.csv)
data = pd.read_csv('data/bank.csv')

In [ ]:
# Versión numérica de la columna "deposit"
data["deposit_binary"] = data["deposit"].replace({"yes": 1, "no": 0})

In [ ]:
# Normalización de las columnas numéricas
numeric_cols = ["balance", "pdays", "previous"]
data[numeric_cols] = (data[numeric_cols] - data[numeric_cols].mean()) / data[numeric_cols].std()

Utilizaremos solo una variable categórica correspondiente a:

housing: tiene préstamo hipotecario? (categórica: 'no', 'yes')

Para esto la codificamos primero:

In [ ]:
# Versión numérica de la columna poutcome
data["housing_numeric"] = data["housing"].map({"yes": 1, "no": 0})

El modelo seguirá siendo básicamente el mismo, nada más que $\alpha$ dependerá de la variable poutcome.

$$
\begin{align}
\begin{array}{lcl}
deposit_i & \sim & \text{Bernoulli}(p_i) \\
\text{logit}(p_i) & = & \alpha_{POUTCOME[i]} + \beta_{balance}balance_i + \beta_{pdays}pdays_i + \beta_{previous}previous_i \\
\alpha_j & \sim & \text{Normal}(0, 1) \qquad \text{for } j=0,1,2,3\\
\beta_{balance} & \sim & \text{Normal}(0, 1) \\
\beta_{pdays} & \sim & \text{Normal}(0, 1) \\
\beta_{previous} & \sim & \text{Normal}(0, 1) \\
\end{array}
\end{align}
$$

In [ ]:
# Partición de los datos en entrenamiento y prueba
X = data[["balance", "pdays", "previous", "housing_numeric"]]
y = data["deposit_binary"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Redefinimos modelo con variables mutables
coords = {"cat": X_train["housing_numeric"].unique()}
with pm.Model(coords=coords) as categorical_model:
    # Variables numéricas mutables
    balance = pm.MutableData("balance", X_train["balance"].values, dims="idx")
    pdays = pm.MutableData("pdays", X_train["pdays"].values, dims="idx")
    previous = pm.MutableData("previous", X_train["previous"].values, dims="idx")
    housing = pm.MutableData("housing", X_train["housing_numeric"].values, dims="idx")
    deposit_data = pm.MutableData("deposit_data", y_train.values, dims="idx")
    # Previas
    alpha = pm.Normal("alpha", mu=0, sigma=1, dims="cat")
    beta_balance = pm.Normal("beta_balance", mu=0, sigma=1)
    beta_pdays = pm.Normal("beta_pdays", mu=0, sigma=1)
    beta_previous = pm.Normal("beta_previous", mu=0, sigma=1)
    # Regresión
    p = pm.Deterministic(
        "p",
        pm.math.invlogit(
            alpha[housing]
            + beta_balance * balance
            + beta_pdays * pdays
            + beta_previous * previous
        ),
    )
    # Deposit
    deposit = pm.Bernoulli("deposit", p=p, observed=deposit_data, dims="idx")
    
    # Muestreo de la distribución posterior
    idata = pm.sample()

In [ ]:
# az.plot_posterior
az.plot_posterior(idata, var_names=["beta_balance", "beta_pdays", "beta_previous"])

In [ ]:
# Importar scipy.special.expit as logistic
from scipy.special import expit as logistic

In [ ]:
# Plot posterior
az.plot_posterior(
    idata,
    var_names=["alpha"],
    transform=logistic,
)

In [ ]:
# Cambiamos los datos observados y muestreamos la distribución posterior predictiva
with categorical_model:
    pm.set_data(
        {
            "balance": X_test["balance"].values,
            "pdays": X_test["pdays"].values,
            "previous": X_test["previous"].values,
            "housing": X_test["housing_numeric"].values,
            "deposit_data": y_test.values
        },
        coords={"idx": X_test.index}
    )

    idata.extend(pm.sample_posterior_predictive(idata))

In [ ]:
# Tomamos promedio sobre las cadenas y las muestras
y_score = idata.posterior_predictive["deposit"].mean(dim=["chain", "draw"])
y_score

In [ ]:
# Construimos la curva precision-recall
precision, recall, thresholds = precision_recall_curve(y_true=y_test, y_score=y_score)
pr_curve = pd.DataFrame({'Precision': precision, 'Recall': recall})

In [ ]:
# Construimos la curva F1 vs. umbrales
f1 = pd.Series({t: f1_score(y_true=y_test, y_pred=y_score > t) for t in thresholds})
# Encontramos el umbral que maximiza F1
best_threshold = f1.idxmax()

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(13, 5))

# Curva Precision-Recall, con el mejor umbral marcado
sns.scatterplot(x='Recall', y='Precision', data=pr_curve, ax=axes[0])
axes[0].plot(
    pr_curve.loc[np.where(thresholds == best_threshold)[0], 'Recall'],
    pr_curve.loc[np.where(thresholds == best_threshold)[0], 'Precision'],
    'ro',
    ms=10
)
axes[0].set_ylim(0,1)
axes[0].set_title('Precision-Recall Curve')

# Curva F1 vs. umbrales, con el mejor umbral marcado
f1.plot(ax=axes[1], title='F1 Scores', ylim=(0,1))
axes[1].set_xlabel('Threshold')
axes[1].axvline(best_threshold, lw=1, ls='--', color='k')
axes[1].text(x=.60, y=.95, s=f'Max F1 @ {best_threshold:.2f}')
fig.tight_layout()
plt.subplots_adjust(top=.8)
plt.show();

In [ ]:
# Accuracy
accuracy = ((y_score > best_threshold) == y_test).mean()
accuracy

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>